# １．　クローラーの設計の基本

## クローラーの種類
目的と対象に応じて必要十分な機能のみを持ったクローラーを開発する

## 目的と対象を明確にする
* 目的が「あるサイトの画像データをすべてダウンロードする」であれば、wgetで十分  
* 対象のサイトのURLがaタグのhref属性に入っておらず、data-url属性にしか入っていない場合はwgetでは難しい  
* 新着情報があったときにメールやSlackで通知を送りたい場合もwgetでは難しい

## URL構造を確認する
* 対象のサイトのサイトマップを確認する
* サイトマップをXMLで提供しているサイトもある
* サイトマップが得られない場合は、カテゴリ一覧のページがないかサイト内を探索する

## 目的とするデータの提供がないか確認する
* サイトによってはクローラーによるサーバーへのアクセス負荷を避けるために、公式でアーカイブデータを提供している場合もある（Wikipediaなど）
* サイト上にクローラーアクセスに関する注意が書かれていることもあるため、注意事項をよく読む

## Web API
* Web APIを利用して特定のURLに決められたパラメータでアクセスすれば、XMLやJSONといった構造化されたデータが得られる
* QiitaではWeb APIによるJSON形式のデータと、各タグに紐付けられた投稿のAtomフィードを提供している
* JSON形式のデータはjqなどのJSONユーティリティを使うと見やすくなる


In [1]:
!curl 'https://qiita.com/api/v2/items?page=1&per_page=20' | jq .

\"lazy\"></a><br>\nCSSがHTMLを装飾してwebブラウザに表示します。</p>\n\n<p>実際ブラウザ上でどのような違いがあるのか、CSS有り、無しで表示した画像を並べてみました。</p>\n\n<p><a href=\"https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F792954%2F1fd3bb00-6935-366b-2883-f623f8e0ec49.png?ixlib=rb-1.2.2&amp;auto=format&amp;gif-q=60&amp;q=75&amp;s=ca25d7c087fbba2e797ec1dcb0a5ee8f\" target=\"_blank\" rel=\"nofollow noopener\"><img src=\"https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F792954%2F1fd3bb00-6935-366b-2883-f623f8e0ec49.png?ixlib=rb-1.2.2&amp;auto=format&amp;gif-q=60&amp;q=75&amp;s=ca25d7c087fbba2e797ec1dcb0a5ee8f\" alt=\"スクリーンショット 2021-02-09 18.26.30.png\" data-canonical-src=\"https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/792954/1fd3bb00-6935-366b-2883-f623f8e0ec49.png\" srcset=\"https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F792954%2F1fd3bb00-6935-366b-

# ２．　クローラーの持つ各処理工程ごとの設計と注意点

## 設計の勘所
![](./images/20210208152750.png)

## ネットワークリクエスト
* 間隔を空ける  
少なくとも１秒は空ける

* タイムアウト  
クロールしたタイミングでサーバーが高負荷の場合は、3秒待って応答がなければタイムアウトしたほうが効率的

* リトライ  
リトライは１から３回ぐらいがよい  
Exponential Backoff: リトライまでの最適な時間をシステムが計算すること。  
通信に失敗する回数が多いほど、１秒、２秒、４秒、８秒と２のn乗で秒数を増やす。

## パース
* 文字コード  
HTMLソースの場合は様々な文字コードで書かれているため、抽出結果が文字化けすることがある  
サイトごとの文字コードを予め調べておき、決め打ちで処理するのが最も良い  
それができない場合は文字コード変換ライブラリを使う  

* HTML/XML解析  
HTML Tidy: HTMLのタグ欠けなどを修復してくれるライブラリ  
Beautiful Soup: Pythonの代表的なHTML/XML解析プログラム  
lxml: Pythonの代表的なHTML/XML解析プログラム 

* JSONデコーダー  
JSONデコーダーを使うことで辞書型（連想配列型）のデータに変換できる

## スクレイピングと正規表現
* URL正規化  
相対リンクは絶対リンクに修正しておく  

* テスト  
テストコードを書くことで収集処理との分離がしやすくなり、目視での確認を毎回行わずに済む

## データストレージの選定と構造
* ファイル  
一つのディレクトリに数万のファイルを保存すると、ファイル一覧などを取得する処理で不具合が生じることがあるが、ディレクトリを分散させることで不具合を避けることができる  
「ダウンロードしたファイルがファイルがどのページに紐づくか」などの属性的な情報が不要で、ファイル数が数万を超えてくるようであれば、保存ファイル名をSHA-1形式にハッシュ化し、先頭２文字でディレクトリを分けるなど工夫をする  

* データベース  
ダウンロードしたファイルがファイルがどのページに紐づくか、リクエスト時のレスポンスヘッダーがどうであったかなど、付帯情報を細かく記録、保存していく場合はMySQLなどのリレーショナル・データベースが向いている  

* データの存在確認  
すでに対象URLが保存済みの場合はリクエストしないようにする

# ３．　バッチ作成の注意点

## バッチ（バッチプログラム）とは
予め定められた一連の処理の流れを一度に行うこと  

* 工程ごとの分離  
ネットワークリクエストとスクレイピング部分は別の関数やクラスになるように、各工程が分離しているのが理想的  

* 中間データを保存しておく  
各工程を分割するためにも、ネットワークリクエストしたデータは保存しておいたほうがよい  

* 実行時間と経過がわかるようにする  
実行時間がわかるとどこかのサイトでエラーが起きているかもしれないといった判断ができる  
現在処理中のURLや工程のラベル（クローリングの経過）を表示することで、思ったとおりにプログラムが動いているか確認できる  
HTTPステータスを保存しておけば、対象データが保存できなかった場合にリンク切れだったのか、サーバーエラーだったのか、メンテナンス中だったのか判断できる  

* 停止条件を明確にする  
無限ループしないように、クロールする階層やドメインを限定したり、ダウンロードするファイル数の上限を決め、条件に応じてクロールを終了するようにしておく  

* 関数の引数をシンプルにする  
中間データを保存しておき、そのキーを引数にするとよい  

* 日時の扱いに注意する  
データベースの更新日時を保存しておく場合、UTC（Coodinated Universal Time）に統一しておくとよい  
クロールを実行するマシンの日時もNTP（Network Time Protocol）を入れて正確な時刻を持つようにする

## 設計の例

### ソースを確認する

In [5]:
!curl -Ss 'https://www.aozora.gr.jp/index_pages/person1562.html#sakuhin_list_1' | head

<html lang="ja">
<head>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8">
<meta http-equiv="Content-Style-Type" content="text/css">
<title>作家別作品リスト：吉川 英治</title>
<style type="text/css">
<!--
body{
    margin-left: 10%;
    margin-right: 10%;


### データの保存方法
データベースに保存する場合は、絵文字などの４バイト文字に対応できるようにしておくと良い  
例えば、MySQLの場合はcharset=utf8mb4で指定する

### ファイルの保存形式
* CSV: Comma-Separated Values, カンマで区切ったデータ. 入れ子構造は持てない
* TSV: Tab-Separated Values, タブで区切ったデータ. 入れ子構造は持てない
* JSON: JavaScript Object Notation, 軽量なデータ交換フォーマット. 入れ子構造を持てる.   
ある程度人間にも読めて、プログラムからも扱いやすい形式としてJSONは悪くない選択肢と言える